In [ ]:
import sys
sys.path.append('..')

# Burn Tiff Labels for SpaceNet Planet Imagery

In [11]:
#https://gist.github.com/avanetten/b295e89f6fa9654c9e9e480bdb2e4d60
from osgeo import gdal, ogr

def create_building_mask(rasterSrc, vectorSrc, npDistFileName='', noDataValue=0, burn_values=1):
    '''
    Create building mask for rasterSrc,
    Similar to labeltools/createNPPixArray() in spacenet utilities
    Args:
        - rasterSrc : str, base GeoTiff location
        - vectorSrc : str, geoJSON to burn
        - npDistFileName: str, output GeoTiff filename
    '''
    
    ## open source vector file that truth data
    source_ds = ogr.Open(vectorSrc)
    source_layer = source_ds.GetLayer()

    ## extract data from src Raster File to be emulated
    ## open raster file that is to be emulated
    srcRas_ds = gdal.Open(rasterSrc)
    cols = srcRas_ds.RasterXSize
    rows = srcRas_ds.RasterYSize

    ## create First raster memory layer, units are pixels
    # Change output to geotiff instead of memory 
    memdrv = gdal.GetDriverByName('GTiff') 
    dst_ds = memdrv.Create(npDistFileName, cols, rows, 1, gdal.GDT_Byte, options=['COMPRESS=LZW'])
    dst_ds.SetGeoTransform(srcRas_ds.GetGeoTransform())
    dst_ds.SetProjection(srcRas_ds.GetProjection())
    band = dst_ds.GetRasterBand(1)
    band.SetNoDataValue(noDataValue)    
    gdal.RasterizeLayer(dst_ds, [1], source_layer, burn_values=[burn_values])
    dst_ds = 0

In [ ]:
from src.datasources import df_spacenet7
import os, tqdm
import pandas as pd
df_planet = df_spacenet7.loc['planet']

## Burn Cloud Masks

In [48]:
df_planet_cloud = pd.DataFrame(list(df_planet['cloud_mask_path']))
for output_filename in tqdm.tqdm(list(df_planet_cloud[0])):
    geojson_path = output_filename.replace("UDM_masks","labels").replace(".tif","_UDM.geojson")
    image_path = output_filename.replace("UDM_masks","images")
    create_building_mask(rasterSrc=image_path, vectorSrc=geojson_path, npDistFileName=output_filename)

100%|██████████| 1423/1423 [00:37<00:00, 37.92it/s]


## Burn Building Masks

In [13]:
df_planet_building = pd.DataFrame(list(df_planet['label_mask_path']))
for output_filename in tqdm.tqdm(list(df_planet_building[0])):
    geojson_path = output_filename.replace("masks","labels").replace(".tif",".geojson")
    image_path = output_filename.replace("masks","images").replace("_Buildings","")
    create_building_mask(rasterSrc=image_path, vectorSrc=geojson_path, npDistFileName=output_filename)

100%|██████████| 1423/1423 [03:24<00:00,  6.97it/s]
